# Dome Azimuth From Telescope Orientation

In [1]:
%matplotlib inline
import numpy as np
import numpy.linalg as la

# Plotting
# import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns

# Either one of the following imports needs to be used! (1) Linux (2) W10
# from matplotlib_inline.backend_inline import set_matplotlib_formats
from IPython.display import set_matplotlib_formats

from mpl_toolkits.mplot3d import Axes3D
# from pytransform3d.transformations import transform_from, plot_transform, concat
# from pytransform3d.rotations import random_axis_angle, matrix_from_axis_angle

# Plotting settings
set_matplotlib_formats('png', 'pdf')

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

sns.set_context("notebook", font_scale=1.3, rc={"lines.linewidth": 2.5})

# For astro conversions
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u
import datetime

*pytransform3d example:*

In [2]:
# A2B = transform_from(
#     R=matrix_from_axis_angle(np.array([1, 0, 0, np.pi/2])),
#     p=np.array([10,0,0]))
# A2B @ np.array([1, 1, 1, 1])

## Modelling The Telescope Pointing

Relevant dimensions/constants (in meters unless otherwise stated):

In [3]:
# Telescope:
l_1 = 0.8
l_2 = 0.7
l_3 = 0.3

# Dome:
d = 6.5 # diameter
r = d/2 # radius
e = 1.6 # extent of cylindrical dome wall
h = 4.9 # height of half capsule repr. dome
w = 1.6 # Slit width

# Obvervatory:
lat = 53.24 # degrees

Defining the transformation matrices:

In [4]:
def vec(x, y, z):
    return np.array([x, y, z, 1]).reshape((4,1))

def trans(x, y, z):
    return np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1],
    ])

def rot_x(angle):
    angle = np.radians(angle)
    return np.array([
        [1,             0,              0, 0],
        [0, np.cos(angle), -np.sin(angle), 0],
        [0, np.sin(angle),  np.cos(angle), 0],
        [0,             0,              0, 1],
    ])

def rot_y(angle):
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle),  0,  -np.sin(angle), 0],
        [0,              1,               0, 0],
        [-np.sin(angle), 0,   np.cos(angle), 0],
        [0,              0,               0, 1],
    ])

def rot_z(angle):
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle),  -np.sin(angle), 0, 0],
        [np.sin(angle),  np.cos(angle), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ])

Compute the position of the telescope back plate with `get_transform` and `telescope_pos`.

In [5]:
def get_transform(ha, dec, y_back=-0.5):
    H_01 = trans(0, 0, l_1)
    H_12 = rot_x(lat) @ rot_z(ha) @ trans(0, 0, l_2)
    H_23 = rot_x(dec) @ trans(-l_3, 0, 0)
    H_34 = trans(0, y_back, 0)
    
    H = H_01 @ H_12 @ H_23 @ H_34
    
    return H

def telescope_pos(ha=0, dec=0):
    """
    Compute the position of the
    telescope in the dome.
    """
    origin = vec(0, 0, 0)
    pose_matrix = get_transform(ha, dec, y_back=0)
    
    return pose_matrix @ origin

## Telescope in dome frame

Compute the origin of each consecutive reference frame with `get_origins`.

In [6]:
def get_origins(ha, dec, y_back=-0.3):
    """Compute the origin of each intermediate ref. frame."""
    origin = vec(0, 0, 0)
    H_01 = trans(0, 0, l_1)
    H_12 = rot_x(lat) @ rot_z(ha) @ trans(0, 0, l_2)
    H_23 = rot_x(dec) @ trans(-l_3, 0, 0)
    H_34 = trans(0, y_back, 0)
    H_45 = trans(0, -3*y_back, 0)
    
    pose_0 = origin # Dome origin
    pose_1 = H_01 @ origin # RA axis origin
    pose_2 = H_01 @ H_12 @ origin # Dec axis origin
    pose_3 = H_01 @ H_12 @ H_23 @ origin # Center aperture
    pose_4 = H_01 @ H_12 @ H_23 @ H_34 @ origin # Back aperture
    pose_5 = H_01 @ H_12 @ H_23 @ H_34 @ H_45 @ origin # Front aperture

    return np.array([pose_0, pose_1,pose_2,pose_3,pose_4, pose_5])

def get_direction(p1, p2):
    d = p2 - p1
    d_unit = d/la.norm(d)
    
    return d_unit

To calculate the dome azimuth, using `atan2`, we require a conversion function. Assuming we assume the azimuth to be 0 north and increasing in the eastward direction.

In [7]:
def compute_azimuth(x, y):
    """
    Return the azimuth (rad) using
    the north clockwise conv.
    """
    az = np.arctan2(x, y) # note (x,y) rather than (y,x)
    
    if az < 0:
        az += 2*np.pi
    
    return az

## Dome Azimuth

In [8]:
class Capsule:
    def __init__(self, radius, extent, center_x=0, center_y=0, center_z=0):
        self.r = radius
        self.e = extent
        self.c = np.array([center_x, center_y, center_z])
        
    def get_radius(self):
        return self.r
    
    def get_extent(self):
        return self.e
    
    def get_center(self):
        return self.c
    
class Vector:
    def __init__(self, x=0, y=0, z=0):
        self.x = x
        self.y = y
        self.z = z
        self.array = np.array([x, y, z])
        
    def get_x(self):
        return self.x
        
    def get_y(self):
        return self.y
        
    def get_z(self):
        return self.z
    
    def get_array(self):
        return self.array

In [9]:
def find_intersection(point, direction, capsule):
    has_intersection = False
    t = None
    
#     if direction.z < 0:
#         return has_intersection, t
    
    if np.isclose(direction.x, 0) and np.isclose(direction.y, 0):
        z = e + np.sqrt(capsule.r**2 - point.x**2 - point.y**2)
        t = z - point.z
        
        has_intersection = True
        return has_intersection, t
    
    # If the direction vector is not (nearly) parallel to 
    # the z-axis of the capsule
    a2 = direction.x**2 + direction.y**2
    a1 = point.x*direction.x + point.y*direction.y
    a0 = point.x**2 + point.y**2 - capsule.r**2
    
    delta = a1**2-a0*a2
    t     = (-a1+np.sqrt(delta))/a2
    
    if point.z + t * direction.z >= capsule.e:
        a0 = point.x**2 + point.y**2 + (point.z-capsule.e)**2 - capsule.r**2
        a1 = point.x*direction.x + point.y*direction.y + (point.z-capsule.e)*direction.z
        
        t = -a1+np.sqrt(a1**2-a0)
    
    if t:
        has_intersection = True
    
    return has_intersection, t

def get_ray_intersection(p, d, t):
    return p.get_array() + t*d.get_array()

## Telescope-Dome Model

**Note:** To use the nort-clockwise convension to calculate the azimuth angle, use `atan2(x, y)` rather than `atan2(y, x)`, which gives the angle in an east-counterclockwise manner.

In [10]:
ha = np.linspace(-180, 180, 100)
dec = np.linspace(-90, 90, 100)

In [11]:
def get_dome_azimuth(ha, dec, dome, print_result=False):
    """
    Using the HA/Dec angles, compute
    the azimuth of the dome.
    
    Returns a tuple (az, STATUS):
    - STATUS = 0 -> Error
    - STATUS = 1 -> Succes
    """
    az    = 0
    STATUS = 1

    # Get the origins of each ref. frame 
    os = get_origins(ha, dec)[:,:3].reshape(6, 3)

    # Compute dome intersection
    tube_center = os[[3,5]][0]
    direction = Vector(*get_direction(tube_center, os[[3,5]][1]))
    p = Vector(*tube_center)

    try:
        has_intersection, t = find_intersection(p, direction, dome)

        if has_intersection:
            p_s = get_ray_intersection(p, direction, t)
            os = np.concatenate([os, [p_s]])

            az = compute_azimuth(p_s[0], p_s[1])
            
            if print_result:
                print('Az =', np.degrees(az),'degrees')
    except:
        STATUS = 0
    
    return np.degrees(az)

get_az = np.vectorize(get_dome_azimuth)

Define the dome and compute az. angles:

In [12]:
dome  = Capsule(r, e)

az_0  = get_az(ha, 0, dome)

az_p_15 = get_az(ha, 15, dome)
az_m_15 = get_az(ha, -15, dome)

az_p_30 = get_az(ha, 30, dome)
az_m_30 = get_az(ha, -30, dome)

In [14]:
fig = plt.figure(figsize=(10, 6))
frame = fig.add_subplot(1, 1, 1)

frame.plot(ha, az_0, color='black', label=r'$\delta=0$ deg')

frame.plot(ha, az_p_15, ls='--', color='black', label=r'$\delta=15$ deg')
frame.plot(ha, az_m_15, ls='--', color='grey', label=r'$\delta=-15$ deg')

frame.plot(ha, az_p_30, ls='-.', color='black', label=r'$\delta=30$ deg')
frame.plot(ha, az_m_30, ls='-.', color='grey', label=r'$\delta=-30$ deg')

frame.set_xlabel(r'HA [deg]', fontsize=18)
frame.set_ylabel(r'Dome azimuth [deg]', fontsize=18)
frame.grid(ls='--', alpha=0.5)

frame.set_ylim(0, 360)
frame.set_xlim(-180, 180)

frame.legend(fontsize=16)

# fig.savefig('images/dome_az.png', dpi=150)
plt.show()

RuntimeError: dvipng was not able to process the following string:
b'$\\\\mathdefault{-150}$'

Here is the full report generated by dvipng:
This is dvipng 1.16 Copyright 2002-2015, 2019 Jan-Ake Larsson
[1] 



<Figure size 720x432 with 1 Axes>